In [1]:
import sys
from tello import Tello
import cv2
import numpy as np
import jetson.inference
import jetson.utils
from matplotlib import pyplot as plt
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

In [2]:
# screen width and height
w,h = 240, 180

# PID controller parameters
#kp ,kd ,ki
#pid=[0.2,0.2,0]
pid=[0.5,0.5,0]
pError=0

In [3]:
drone_busy=False

In [4]:
# create Tello drone instance
me =Tello(tello_ip='192.168.42.121')

# set yaw_velocity default value to 0
me.yaw_velocity=0

# connect to drone
me.connect()
# print drone state and battery information
# state information contains battery information
print(me.get_battery())
print(me.get_current_state())

# start stream for getting video
me.stream_on()

2022-02-16 17:25:39,678 [INFO] tello.py - Tello instance was initialized. tello_ip: '192.168.42.121'. Port: '8889'.
2022-02-16 17:25:39,786 [INFO] tello.py - SDK mode successfully started
2022-02-16 17:25:39,890 [INFO] tello.py - SDK version: 30
2022-02-16 17:25:40,094 [INFO] tello.py - Hardware: TELLO
2022-02-16 17:25:40,199 [INFO] tello.py - Battery percentage: 100


100
{'mid': -1, 'x': -100, 'y': -100, 'z': -100, 'mpry': '0,0,0', 'pitch': 0, 'roll': 0, 'yaw': 0, 'vgx': 0, 'vgy': 0, 'vgz': 0, 'templ': 64, 'temph': 67, 'tof': 10, 'h': 0, 'bat': 100, 'baro': 187.2, 'time': 0, 'agx': 0.0, 'agy': 2.0, 'agz': -999.0}


2022-02-16 17:25:40,508 [INFO] tello.py - Video stream enabled


In [5]:
# create network instance of SSD-mobilenet-v2 network with precision threshold equal to 0.5
net = jetson.inference.detectNet("ssd-mobilenet-v2", threshold=0.5)

In [6]:
# this function stops the drone's yaw movement
# when no person is detected
def stop_tracking(myDrone):

# set yaw_velocity to 0
	myDrone.yaw_velocity = 0

# send the set value to drone via Tello's joystick_control command
	send_command("joystick_control",myDrone.yaw_velocity)#myDrone.joystick_control(0,0,myDrone.yaw_velocity,0)

In [7]:
# this function receives drone instance, information about center coordinates (x,y) of person, pid k values
# and the last error value, calculas yaw speed and new error, sends this yaw speed to drone (or stop movement
# signal, in case of missing detection), and returns new error value back to calling thread
def trackPerson(myDrone, info, w, pid, pError):
	#print("start tracking")
	# calculate new PID error and speed based on the center x value of detected Person and the center of image
	error=info[0] -w//2
	speed=pid[0]*error+pid[1]*(error-pError)
	speed=int(np.clip(speed,-100,100))

	print(speed)

	# if there is a detection, value of center is different from 0
	if info[0]!=0:
		myDrone.yaw_velocity= speed
	else:
	# in case of missing detection stop moving in yaw space
		myDrone.yaw_velocity = 0
		error=0
	print("sending yaw velocity, ", myDrone.yaw_velocity)
	# chack for possibility of sending joystick_control command and send the calculated speed
	#if myDrone.joystick_control:
	#send_command("joystick_control",0)
	send_command("joystick_control",speed)#myDrone.yaw_velocity)#myDrone.joystick_control(0,0,myDrone.yaw_velocity,0)
	# return error for the next iteration
	return error


In [8]:
def send_command(command, arg=0):
    global drone_busy
    if command=="move_left":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_left(arg)
                drone_busy=False
                break
    elif command=="move_right":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_right(arg)
                drone_busy=False
                break
    elif command=="move_up":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_up(arg)
                drone_busy=False
                break
    elif command=="move_down":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_down(arg)
                drone_busy=False
                break
                
    elif command=="move_forward":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_forward(arg)
                drone_busy=False
                break
    elif command=="move_backward":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_backward(arg)
                drone_busy=False
                break
    elif command=="rotate_counterclockwise":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.rotate_counterclockwise(arg)
                drone_busy=False
                break
    elif command=="rotate_clockwise":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.rotate_clockwise(arg)
                drone_busy=False
                break
    elif command=="land":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.land()
                drone_busy=False
                break
    elif command=="takeoff":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.takeoff()
                drone_busy=False
                break
    elif command=="joystick_control":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.joystick_control(0,0,0,0)
                me.joystick_control(0,0,arg,0)
                drone_busy=False
                break
    drone_busy=False
    
                

In [9]:
# this function uses keyboard press information from OpenCV to send basic movement commands to Tello
def navigate3(key):
    print(key)
    # using global drone value
    global me
    global drone_busy
    # setting distance (in cm) and angle (in degrees) values for movements
    distance=30
    angle=20

    # checking for code and sending the command
    if key=="left" : send_command("move_left",distance)
    elif key=="right" :send_command("move_right",distance)

    if key=="forward": send_command("move_forward",distance)
    elif key=="back": send_command("move_backward",distance)

    if key=="w": send_command("move_up",distance)
    elif key=="s": send_command("move_down",distance)

    if key=="a": 
        print("a rotate counter clockwise")
        send_command("rotate_counterclockwise",angle)
    elif key=="d": send_command("rotate_clockwise",angle)

    if key=="land": send_command("land")
    # some commands were deactivated and being tested right now
    #if kp.getKey("l"): me.go_xyz_speed(0,0,10,10)
    if key=="to": send_command("takeoff")
    #if kp.getKey("1"): me.send_command_with_return('downvision 0')
    #if kp.getKey("2"): me.send_command_with_return('downvision 1')

In [10]:
# this function uses keyboard press information from OpenCV to send basic movement commands to Tello
def navigate2(key):
    print(key)
    # using global drone value
    global me
    global drone_busy
    # setting distance (in cm) and angle (in degrees) values for movements
    distance=30
    angle=20

    # checking for code and sending the command
    if key=="left" : me.move_left(distance)
    elif key=="right" : me.move_right(distance)

    if key=="forward": me.move_forward(distance)
    elif key=="back": me.move_backward(distance)

    if key=="w": me.move_up(distance)
    elif key=="s": me.move_down(distance)

    if key=="a": 
        print("a rotate counter clockwise")
        me.rotate_counterclockwise(angle)
    elif key=="d": me.rotate_clockwise(angle)

    if key=="land": me.land()
    # some commands were deactivated and being tested right now
    #if kp.getKey("l"): me.go_xyz_speed(0,0,10,10)
    if key=="to": me.takeoff()
    #if kp.getKey("1"): me.send_command_with_return('downvision 0')
    #if kp.getKey("2"): me.send_command_with_return('downvision 1')

In [11]:
# this function uses keyboard press information from OpenCV to send basic movement commands to Tello
def navigate(key):
# the codes of keys from opencv
# 119 w
# 101 e
# 100 d
# 97 a
# 115 s
# 104 h
# 106 j
# 107 k
# 108 l
# 113 q
# 114 r
# 101 e

# using global drone value
	global me
# setting distance (in cm) and angle (in degrees) values for movements
	distance=30
	angle=20

	# checking for code and sending the command
	if key==104 : me.move_left(distance)
	elif key==106 : me.move_right(distance)

	if key==107: me.move_forward(distance)
	elif key==108: me.move_backward(distance)

	if key==119: me.move_up(distance)
	elif key==115: me.move_down(distance)

	if key==97: me.rotate_counterclockwise(angle)
	elif key==100: me.rotate_clockwise(angle)
	
	if key==114: me.land()
	# some commands were deactivated and being tested right now
	#if kp.getKey("l"): me.go_xyz_speed(0,0,10,10)
	if key==101: me.takeoff()
	#if kp.getKey("1"): me.send_command_with_return('downvision 0')
	#if kp.getKey("2"): me.send_command_with_return('downvision 1')

In [12]:
image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(me.read_frame())




In [13]:
def update_image(frame):
    image_widget.value = bgr8_to_jpeg(frame)

In [14]:
distance=30
angle=20

In [15]:
def send_to_navigate2(inp):
    #print(inp)
    navigate3(inp.description.lower())

In [16]:
buttonW=ipywidgets.Button(description='W')
buttonW.on_click(send_to_navigate2)
buttonS=ipywidgets.Button(description='S')
buttonS.on_click(send_to_navigate2)
buttonA=ipywidgets.Button(description='A')
buttonA.on_click(send_to_navigate2)
buttonD=ipywidgets.Button(description='D')
buttonD.on_click(send_to_navigate2)


buttonU=ipywidgets.Button(description='FORWARD')
buttonU.on_click(send_to_navigate2)
buttonDown=ipywidgets.Button(description='BACK')
buttonDown.on_click(send_to_navigate2)
buttonL=ipywidgets.Button(description='LEFT')
buttonL.on_click(send_to_navigate2)
buttonR=ipywidgets.Button(description='RIGHT')
buttonR.on_click(send_to_navigate2)
buttonland=ipywidgets.Button(description='LAND')
buttonland.on_click(send_to_navigate2)
buttontakeoff=ipywidgets.Button(description='TO')
buttontakeoff.on_click(send_to_navigate2)

In [17]:
all_widget = ipywidgets.VBox([
    #image_widget,
    ipywidgets.HBox([ipywidgets.VBox([buttonW,
    ipywidgets.HBox([buttonA, buttonD]),buttonS
]), ipywidgets.VBox([buttonU,
    ipywidgets.HBox([buttonL, buttonR]),buttonDown
]), ipywidgets.VBox([buttonland,buttontakeoff
])]),
])

#display(all_widget)

In [18]:
import threading

In [19]:
def detect_camera():
    global pError
    global drone_busy
    while True:
        # read frame from Tello
        frame=me.read_frame()


        if (frame is not None):
            # resize frame image to width and height, given above
            img = cv2.resize(frame, (w, h))
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # change the image format from Numpy array to CUDA (used for jetson detection)
            cuda_image=jetson.utils.cudaFromNumpy(img)
            # detect objects from CUDA image
            detections = net.Detect(cuda_image)

            # flag for checking if object was found
            object_found = False
            # list for center coordinates of object
            objectsListC = []
            # list for area values of detections
            objectsArea = []
            # iterate over detections
            for detection in detections:
                # get class name from ID
                cl_name=net.GetClassDesc(detection.ClassID)
                # get detection center coordinates
                center=detection.Center
                # check for class name, only person is required in this implementation
                if (cl_name=="person"):
                    # set the flag to true
                    object_found=True
                    # print detection values
                    #print("Top: {}, Bottom: {}, Left: {}, Right: {}, Height: {}, Width: {}, Area: {}, Center: {}, ".format(detection.Top,detection.Bottom,detection.Left,detection.Right,detection.Height,detection.Width, detection.Area,detection.Center))
                    # append found detection center and area to responding lists
                    objectsListC.append(detection.Center)
                    objectsArea.append(detection.Area)
            # if there were detections found
            #stop_tracking(me)
            if len(objectsArea) != 0:
                # get the center value with maximum area
                i = objectsArea.index(max(objectsArea))
                # send tracking command and update PID error value
                pError = trackPerson(me, objectsListC[i], w, pid, pError)
            # if object was not found
            #if object_found == False:
            #stop drone movement
                #stop_tracking(me)

            # test code for understanding detection objects inside structure
            #print(dir(detection))
            #['Area', 'Bottom', 'Center', 'ClassID', 'Confidence', 'Contains', 'Height', 'Instance', 'Left', 'ROI', 'Right', 'Top', 'Width', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

            #Top: 214.1015625, Bottom: 655.6640625, Left: 672.1875, Right: 841.875, Height: 441.5625, Width: 169.6875, Area: 74927.640625, Center: (757.03125, 434.8828125), 

            # convert the cuda image with detections on it back to numpy image
            nmp_img=jetson.utils.cudaToNumpy(cuda_image)
            # show that numpy image on desktop
            #cv2.imshow('image', nmp_img)
            #nmp_img== cv2.cvtColor(nmp_img, cv2.COLOR_BGR2GRAY)

            update_image(nmp_img)
            #_, encoded=cv2.imencode('.png',nmp_img)[1]
            #im_bytes=nmp_img.tobytes()
            #base64String=base64.b64encode(im_bytes)#encoded.tostring().encode('base64')
            #display(HTML('&amp;amp;lt;img src="data:image/png;base64,%s" width=300px&amp;amp;gt;&amp;amp;lt;/img&amp;amp;gt;'%base64String))
            # opencv wait for keyboard press
            res = cv2.waitKey(1)
            # if (res!=-1):
            # 	print('You pressed %d (0x%x), LSB: %d (%s)' % (res, res, res % 256,
            # repr(chr(res%256)) if res%256 < 128 else '?'))
            # if q was pressed land drone and stop loop execution
            if(chr(res%256)=='q'):
                me.land()
                break
            # else use the given command to move the drone
            navigate(res)

In [20]:
new_thread = threading.Thread(target=detect_camera)
new_thread.start()

In [21]:
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [22]:
display(all_widget)

In [41]:
btn = ipywidgets.Button(description="Save")
btnA = ipywidgets.Button(description="A")
btnB = ipywidgets.Button(description="B")
btnC = ipywidgets.Button(description="C")
box_layout = ipywidgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='50%')
box = ipywidgets.GridBox(children=[btn],
        layout=ipywidgets.Layout(
            width='50%',
            grid_template_columns='100px 50px 100px 200p',
            grid_template_rows='80px auto 80px',
            grid_gap='25px 50px')
       )
display(box)

GridBox(children=(Button(description='Save', style=ButtonStyle()),), layout=Layout(grid_gap='5px 10px', grid_t…

In [40]:
[ipywidgets.Button(layout=ipywidgets.Layout(width='auto', height='auto'),
                         style=ipywidgets.ButtonStyle(button_color='darkseagreen')) for i in range(9)]

[Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen')),
 Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_color='darkseagreen'))]

In [ ]:
all_widget = ipywidgets.VBox([
    #image_widget,
    ipywidgets.HBox([ipywidgets.VBox([buttonW,
    ipywidgets.HBox([buttonA, buttonD]),buttonS
]), ipywidgets.VBox([buttonU,
    ipywidgets.HBox([buttonL, buttonR]),buttonDown
]), ipywidgets.VBox([buttonland,buttontakeoff
])]),
])